### It turns out that Epoch = 3, Batch Size = 8 , Learning Rate = 5e-5 performs well (without overfitting)

In [1]:
import numpy as np
import time
import json
import os
import pandas as pd
import re

from datasets import Dataset
from transformers import AutoTokenizer
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification
from transformers import DefaultDataCollator

/home/lee/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Data into Pandas DF

In [2]:
## Load data

TRAIN_DATA_PATH = "./full_data/data_storage/full_train.json"
Y_TRAIN_ID_PATH = "./id_data/train.label.txt"
X_TRAIN_ID_PATH = "./id_data/train.data.txt"   # train

DEV_DATA_PATH = "./full_data/data_storage/full_dev.json"
Y_DEV_ID_PATH = "./id_data/dev.label.txt"
X_DEV_ID_PATH = "./id_data/dev.data.txt"   # dev

TEST_DATA_PATH = "./full_data/data_storage/test_full_v2.json"
X_TEST_ID_PATH = "./id_data/test.data.txt"   # test

In [3]:
## Import Twitter Processer
#!pip install tweet-preprocessor

import preprocessor as p
i = "@Telegraph How \n\n very sad. @Telegraph @Telegraph"
p.clean(i)

'How very sad.'

In [4]:
def convert_replies_id_to_sorted_text(list_of_id, data_df):
    """Convert a list of tweet ids to a list of its corresponding text in data_df
    Sort by tweets original create time"""
    if type(list_of_id) is not list:
        # if input is a single id convert type
        list_of_id = [list_of_id]
    # Select dataframe based on a list of ids
    selected_df = data_df[data_df['id'].astype(str).isin(list_of_id)]
    # Sort by their created time
    sorted_df = selected_df.sort_values(by = ["created_at"])
    # Select text fields only
    text_list = sorted_df["text"].to_list()
    return text_list


def remove_URL(original):
    """Remove url link in the text"""
    result = re.sub(r"http\S+", "", original)
    result = re.sub(r"www.\S+", "", result)
    result = re.sub(r"wasap.my+", "", result)
    return result

def join_data_id_label_v2(LABEL_PATH, ID_PATH, DATA_PATH):
    # Process Labels
    if LABEL_PATH:
        # 1: Rumour
        # 0: NonRumour
        with open(LABEL_PATH, "r") as f:
            y_label = f.read().strip().split("\n") # remove next line
        y_label = pd.DataFrame(y_label, columns = ["label"])
        y_label[y_label["label"]=="rumour"] = 1
        y_label[y_label["label"]=="nonrumour"] = 0

    ## Get Dataframe Id, with first id as source Id, and values as replies, not using dict since we have duplicated keys
    total_id_list = []
    with open(ID_PATH, "r") as f:
        for line in f:
            line = line.strip().split(',') # remove next line
            source_id = line[0]
            if len(line) > 1:
                # if we have replies id
                replies_id = line[1:]
            else:
                replies_id = []
            row = [source_id, replies_id]
            total_id_list.append(row)
    len(total_id_list)
    
    ## Create a dataframe containing a list of replies
    source_df = pd.DataFrame(total_id_list, columns = ['source_id', 'replies_id'])
    data_df = pd.read_json(DATA_PATH)
    source_df["reply_text_list"] = source_df["replies_id"].apply(convert_replies_id_to_sorted_text, data_df = data_df)
    source_df["source_text"] = source_df["source_id"].apply(convert_replies_id_to_sorted_text, data_df = data_df)

    if LABEL_PATH:
      source_df["label"] = y_label
    return source_df

def bert_preprocess(IDS, DATA, LABELS=False):
    """Function to combine all the preprocessing steps"""
    data = join_data_id_label_v2(LABELS, IDS, DATA)
    ## 1. Only keep english tweets as most of them are in english
    # Use only text data and remove URLs
    data["source"] = data["source_text"].apply(" ".join).apply(p.clean) 
    data["replies"] = data["reply_text_list"].apply(" ".join).apply(p.clean)
    
    
    if LABELS:
      data = data[['source','replies','label']]
    else:
      data = data[['source','replies']]
    
    return data

In [5]:
## Keep only source text as a column AND concatenated reply strings as another column
data_train = bert_preprocess(X_TRAIN_ID_PATH, TRAIN_DATA_PATH, LABELS=Y_TRAIN_ID_PATH)
data_train

,source,replies,label
0,. Can regularly rinsing your nose with saline ...,. Can eating garlic help prevent infection wit...,0
1,French police chief killed himself after attack,How very sad. The trauma he must have faced. V...,1
2,Coronavirus disease (COVID-19) advice for the ...,Infection control for suspected or confirmed C...,0
3,Ottawa police confirm that there were multiple...,"Killers go berserk when cornered. Henceforth, ...",0
4,if the primary focus of a government isn't to ...,,0
...,...,...,...
1890,Desperate Ted Cruz Claims Planned Parenthood S...,Desperate! Cruz is one sick dude. Incredibly s...,1
1891,"""Thoughts and prayers are not enough."" Pres. O...",. has anyone else noticed mass shootings skyro...,1
1892,Police have surrounded this building where the...,bury them in their hole didn't the cops ask yo...,0
1893,,Its an insult to God almighty to not take Him ...,0


In [6]:
## Keep only source text as a column AND concatenated reply strings as another column
data_dev = bert_preprocess(X_DEV_ID_PATH, DEV_DATA_PATH, LABELS=Y_DEV_ID_PATH)
data_dev

,source,replies,label
0,COVID-19 Fact:Are hand dryers effective in kil...,"They are, in fact, germ-breeding factories.",0
1,when can we expect the result of my husband's ...,Hi have you had any luck? In the same boat? Ev...,0
2,How does COVID-19 spread? People can catch COV...,I've read a lot about Corona virus lately and ...,0
3,"every news outlet using headlines like,""are an...","Apparently, when a headline is a question, the...",0
4,Researcher on his encounter with a Goliath bir...,eu tenho uma dessas em casa !So timas para coa...,0
...,...,...,...
627,"or cure for COVID-19. However, there are sever...",WHAT ARE THE TREATMENT OPTIONS FOR COVID-19 (I...,0
628,"After speculation that hes been arrested, Bank...",there was a story saying someone had vandalise...,1
629,*Your questions answered**Reply with the numbe...,s?14. Can I catch COVID-19 from infected surfa...,0
630,"Operation Ku Klux Klan, We never stopped watch...",those so called men are hiding something.. tim...,1


## Convert Pandas Dataframe to Transformer DataSet

In [7]:
dataset_train = Dataset.from_pandas(data_train)
dataset_train

Dataset({
    features: ['source', 'replies', 'label'],
    num_rows: 1895
})

In [8]:
dataset_dev = Dataset.from_pandas(data_dev)
dataset_dev

Dataset({
    features: ['source', 'replies', 'label'],
    num_rows: 632
})

## Transformer Dataset to TensorFlow TF Dataset

Makin use of Transformer Dataset

In [9]:
BERT_MODEL = "bert-base-cased"
NUM_EPOCHS = 2
BATCH_SIZE = 8

In [10]:
## Set up Tokenizer


tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL)

## Tokenize with two sentences separated by [SEP]m, use source and reply as two sentences
def tokenize_function(dataset):
    return tokenizer(dataset["source"], dataset["replies"], padding="max_length", truncation=True)

# Set up Train data
tokenized_train_datasets = dataset_train.map(tokenize_function, batched=True)
# Set up Dev Data
tokenized_dev_datasets = dataset_dev.map(tokenize_function, batched=True)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 16.75ba/s]


In [11]:
## Use data_collator to batch the dataset
data_collator = DefaultDataCollator(return_tensors="tf")

In [12]:
tf_train_dataset = tokenized_train_datasets.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["label"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=BATCH_SIZE,
)

tf_validation_dataset = tokenized_dev_datasets.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["label"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=BATCH_SIZE,
)

2022-05-06 22:58:51.214330: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-06 22:58:51.216887: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-06 22:58:51.216963: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-06 22:58:51.217295: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

## Tensor Flow Bert Model

Use Training Set to train and test against dev set

In [13]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [14]:


## Define Model
model = TFAutoModelForSequenceClassification.from_pretrained(BERT_MODEL, num_labels=2)

## Set up optimisation method, minimise which loss
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)


2022-05-06 23:00:14.605606: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
# Fit model
model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=NUM_EPOCHS)


Epoch 1/2


2022-05-06 23:00:31.131201: W tensorflow/core/common_runtime/bfc_allocator.cc:462] Allocator (GPU_0_bfc) ran out of memory trying to allocate 12.00MiB (rounded to 12582912)requested by op gradient_tape/tf_bert_for_sequence_classification/bert/encoder/layer_._11/output/LayerNorm/batchnorm/mul_2/Mul_1
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-05-06 23:00:31.131298: I tensorflow/core/common_runtime/bfc_allocator.cc:1010] BFCAllocator dump for GPU_0_bfc
2022-05-06 23:00:31.131322: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (256): 	Total Chunks: 131, Chunks in use: 129. 32.8KiB allocated for chunks. 32.2KiB in use in bin. 602B client-requested in use in bin.
2022-05-06 23:00:31.131336: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (512): 	Total Chunks: 1, Chunks in use: 0. 512B allocated for ch

ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/tf_bert_for_sequence_classification/bert/encoder/layer_._11/output/LayerNorm/batchnorm/mul_2/Mul_1' defined at (most recent call last):
    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/lee/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/lee/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/home/lee/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/lee/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.1

In [ ]:
import matplotlib.pyplot as plt

train_acc = [0.7791, 0.8983, 0.9465, 0.9714, 0.9783, 0.9836, 0.9825, 0.9878, 0.9889, 0.9809, 0.9889, 0.9380, 0.9539, 0.9836, 0.9931, 0.9952, 0.9931, 0.9921, 0.9793, 0.9905]
dev_acc = [0.8022, 0.9225, 0.9351, 0.9066, 0.9288, 0.9256, 0.9304, 0.9288, 0.9225, 0.9225, 0.9161, 0.9193, 0.9177, 0.9019, 0.9256, 0.9272, 0.9130, 0.9256, 0.9098, 0.9114]

x = range(1, len(train_acc)+1)
plt.rcParams['figure.dpi'] = 200

plt.plot(x, dev_acc)
plt.plot(x, train_acc)
plt.xticks(range(1, len(train_acc)+1))

plt.xlabel("Num of Iterations")
plt.ylabel("Accuracy")

plt.title("Train VS DEV accuracy, Learning_Rate = 5e-5")

## Test on Test Set

In [ ]:
test_df = bert_preprocess(X_TEST_ID_PATH, TEST_DATA_PATH)
test_dataset = Dataset.from_pandas(test_df)  # Convert to Transformer Dataset

In [ ]:
# Convert to Keras input for Bert model
tokenized_test_datasets = test_dataset.map(tokenize_function, batched=True)
tf_test_dataset = tokenized_test_datasets.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=1,
)

In [ ]:
y_pred = model.predict(tf_test_dataset)

In [ ]:
# generate the csv for prediction
def generate_csv(pred, csv_name):
    ids = pd.Index(range(len(pred)), name='Id')
    predictions = pd.DataFrame(pred, index=ids)
    predictions.columns = ['Predicted']
    predictions.to_csv(csv_name)

def model_output_to_label(model_output):
    """Conver the output class of a tensorflow model to label"""
    logit_df = pd.DataFrame(model_output.to_tuple()[0], columns = ["0","1"])
    ## Choose highest logit as the predicted class
    logit_df["label"] = logit_df.apply(lambda x: 0 if x["0"] > x["1"] else 1, axis=1)
    return logit_df["label"]

In [ ]:
labels = model_output_to_label(y_pred)
generate_csv(labels, "./output/vanillabertmodel.csv")